In [1]:
'''
Specifications of Network 1

Single network of 100 neurons (80 excitatory/20 inhibitory)
Random topology, using the simplified neuron model of last year's code
Total run time of 1,000 ms

Display the network model with the functions of the Visualization class
Plot the raster plot of the simulation using the functions of the BrianVisualization class. 

The Sample 1 and Sample 4 code from last year contain most of the code to start this model of a single network.  
Please compare the raster plot of the simulation with and without the Poisson inputs to the neurons.
'''

"\nSpecifications of Network 1\n\nSingle network of 100 neurons (80 excitatory/20 inhibitory)\nRandom topology, using the simplified neuron model of last year's code\nTotal run time of 1,000 ms\n\nDisplay the network model with the functions of the Visualization class\nPlot the raster plot of the simulation using the functions of the BrianVisualization class. \n\nThe Sample 1 and Sample 4 code from last year contain most of the code to start this model of a single network.  \nPlease compare the raster plot of the simulation with and without the Poisson inputs to the neurons.\n"

In [4]:
import networkx as nx
import brian2
import numpy as np

ModuleNotFoundError: No module named 'brian2'

In [ ]:
class Visualization:
    '''
    Function 2: Visualize neural network
    Inputs graph G 
    Returns cluster coefficient & characteristic path length
        & plot of connections between neurons (color-coded)
    For more info: see collective dynamics paper
    
    Description:
    From network model, determines cluster coefficient and characteristic path length for each
        node. For each network, will take average of those values, respectively and yield 
        single integer value.
    From network model, will output plot of connections, color-coded for excitatory and
        inhibitory.
    
    Returns:
        cc_avg: Cluster coefficient averaged over all nodes
        ex_in_plot: Plot of colored excitatory/inhibitory connections
        cpl_avg: Number of edges at shortest path over all nodes 
        
    Parameters:
        G: NetworkX Graph from Function 1
    '''
    def __init__(self):
        #plt.clf() # Clears any previous figures
        plt.close() # Clears any figure windows

    def cluster_coeff(self,G):        
        cc = nx.clustering(G) # calculate clustering co-eff according to different rules eg. no of triangles going through node
        # outputs co-eff for all or specified nodes in dict form
        cc_y=[]
        for idx in cc:
            cc_y=np.append(cc_y,cc[idx]) # access and append dict values (co-effs in this case) to array
        
        cc_avg = np.ndarray.mean(cc_y, dtype=np.float64)
        return cc_avg
    
    def ex_in_connec(self,G,connect_W):
        plt.figure()
        red_patch = mpatches.Patch(color='red', label='Excitatory')
        blue_patch = mpatches.Patch(color='blue', label='Inhibitory')
        plt.legend(handles=[red_patch,blue_patch])
        suptitle('Structural Connections', fontsize=14, fontweight='bold')

        edges = G.edges() # list of tuple pairs
        nodes = G.nodes() # array of nodes

        custom_color={}
        for idx in range(len(connect_W)):
            if connect_W[idx] < 0:
                inhib_edge = new_coord[idx]
                G.add_edge(*inhib_edge)
                custom_color[inhib_edge]='b'
            else:
                excit_edge = new_coord[idx]
                G.add_edge(*excit_edge)
                custom_color[excit_edge]='r'
        if 0:
            for idx,idy in enumerate(edges):
                x1,y1 = edges[idx]
                if connect_W < 0:
                    inhib_edge = (x1,y1)
                    G.add_edge(x1,y1)
                    custom_color[x1,y1]='b' # Stores color of edges in dict
                else:
                    excit_edge = (x1,y1)
                    G.add_edge(x1,y1)
                    custom_color[x1,y1]='r'
        
        ex_in_plot=nx.draw_networkx(G,node_color='w',
                         with_labels=True,
                         node_list=nodes,
                         #node_size=50,
                         node_size=200,
                         edge_list=custom_color.keys(),
                         edge_color=custom_color.values(),
                         label='Blue=Inhibitory, Red=Excitatory')
        #plt.savefig("Structural Connections.png")
        
    def char_path_len(self,G):
        cpl = nx.all_pairs_shortest_path_length(G) # shortest path lengths. Gen. returns tuple with source and target dict
        my_array = []
        my_key = []
        cpl_count = []
        for idx in cpl: # looping through each source node and looking at no of targets and length to target nodes
            myarray = cpl[idx] # cpl is a generator object. idx is a tuple (source, target dict). Should be idx[1].
            min_val = min(ii for ii in myarray if ii > 0) # Find min length
            for key,length in myarray.iteritems():
                if length == min_val:
                    my_key = np.append(my_key,key) # array of target nodes with min length for specific source node
            my_count = len(my_key) # Find number of edges of that length
            cpl_count = np.append(cpl_count,my_count)
            my_key = []
            cpl_avg = np.mean(cpl_count) # Find average of those edges
        return cpl_avg

In [ ]:
class BrianVisualization:
    '''
    Function 4: Visualization of Brian 
    Define LIF neural population in Brian
    Call to save spike times
    Call to plot voltage monitor
    Call to plot raster plot
    Call to plot histogram
    
    Description:
    Will plot the voltage monitor, raster plot, and histogram of neural network
    
    Returns:
        G: NeuronGroup
        spike_times: Spike times for neuron 0
        all_spikes: Spike times for all neurons
        
    
    Parameters:
        statemon: StateMonitor
        spikemon: SpikeMonitor
        run_time: Simulation run time
    
    '''
    def __init__(self):
        plt.clf() # Clears any previous figures
        plt.close() # Clears any figure windows
        
        start_scope()
    
    def network1(self,rows,cols,connect_W,N):
        # rows and cols 1-D arrays of source and target neurons that are connected as defined in graph from networkx
        # connect_W: 1-D array of strength of connections corresponding to source and target neuron pairs 
        '''
        For full synch: G1.v = fixed
                        PI = off
        '''
        eqs = '''                
        dv/dt = (I-v)/tau : 1 (unless refractory)
        I : 1
        tau : second
        '''   # Leaky integrate and fire neuron
                
        G1 = NeuronGroup(N, eqs, threshold='v>v_th', reset='v=v_r', refractory=10*ms, method='linear')
        #G1.v = 'rand()' #random so changes dynamics for each neuron --> causes difference in raster
        G1.v = '0.967188882214'
        '''
        Injection current is constant but with slight perturbations from PoissonInput, if that function is active
        To get rid of highly synchronized, G1.v='rand()' and turn on P1
        '''
        G1.I = I_c # Constant current 
        G1.tau = tau_m1 * ms
        
        # PoissonInput injection current -- changes each neuron's firing rate
        # Each neuron has different input current depending on Poisson distribution
        PI_num = 0.8*N 
        #subG1 = G1[int(PI_num):] # Top 20% of total neurons stimulated
        subG1 = G1[:] # All neurons stimulated via Poisson 
        '''
        PoissonInput(target,target_var,N,rate,weight)
        target: which neurons to send PoissonInput
        target_var: which variable that is being changed from input
        N: number of inputs (more input = higher firing rate)
        rate: rate of input (100Hz = 10ms per spike)
        weight: amount added to voltage
        '''
        #P1 = PoissonInput(subG1, 'v', 5, 100*Hz, weight=0.1) # PoissonInput on
        P1 = PoissonInput(subG1, 'v', 5, 100*Hz, weight=0) # PoissonInput off

        
        S1 = Synapses(G1, G1, 'w : 1', on_pre='v_post += w') # w is the synapse weight added to the signal
        S1.connect(i=rows, j=cols) # Adjacency matrix from Adj.weighted, this uses network structure defined on networkx
        S1.w = connect_W/float(100) # Weighted matrix defined from networkx graph 
                
        return G1,S1,P1
    
    def network2(self,rows,cols,connect_W,N):
        '''
        Start off w/ identical network parameters as network 1, but need to eventually change connect_W (its interconnections)
        If P2 turned on, may need to increase S3.w so network 1 influence is higher than PoissonInput
        '''
        eqs = '''
        dv/dt = (I-v)/tau : 1 (unless refractory)
        I : 1
        tau : second
        '''

        G2 = NeuronGroup(N, eqs, threshold='v>v_th', reset='v=v_r', refractory=10*ms, method='linear')
        #G2.v = '0.967188882214' # For debugging of coupling so that all nodes in G2 will fire at same rate
        G2.v = 'rand()'
        G2.I = I_c
        G2.tau = tau_m2 * ms
        
        subG2 = G2[:]
        P2 = PoissonInput(subG2, 'v', 5, 100*Hz, weight=0.1)
        
        S2 = Synapses(G2, G2, 'w:1', on_pre='v_post += w')
        S2.connect(i=rows, j=cols) # Network 2 has same inter-network connections as Network 1
        S2.w = connect_W/float(100)
        
        return G2,S2,P2

    def network_coupling(self,N,p_couple,w_couple,G1,G2):
        '''
        Should see how coupling between different subpopulation has global effects (raster plot)
            - Could see difference if neurons have same firing rate (non-PoissonInput) vs. different firing rate (all-PoissonInput)
            - May only want to record (Statemon, Spikemon) from this last coupling (G2) to save resources
                - See Monitoring Synaptic Variables from http://brian2.readthedocs.io/en/2.0.1/user/synapses.html
            = Can introduce multiple output synapses (multisynaptic_index from http://brian2.readthedocs.io/en/2.0.1/user/synapses.html)
                - Or more simply "S.connect(i=numpy.arange(10), j=1)"
        '''
        S3 = Synapses(G1,G2, 'w:1', on_pre='v_post += w')#, delay=5*ms) # G1 drives G2
        
        ### Manually defining coupling ###
        p_couple2 = p_couple*N
        i_couple = 0.8*N
        
        # If want 1:1 for only first p_couple% neurons (excitatory --> excitatory)
        c_rows = list(arange(0,p_couple2,dtype=int)) # Source neurons
        c_cols = list(arange(0,p_couple2,dtype=int)) # Target neurons
        
        # If want 1:1 for only last p_couple% neurons 
        #c_rows = list(arange(N-p_couple2,N,dtype=int))
        #c_cols = list(arange(N-p_couple2,N,dtype=int))
        
        # If want 1:1 for inhibitory onto inhibitory neurons
        #c_rows = list(arange(N-i_couple,N,dtype=int))
        #c_cols = list(arange(N-i_couple,N,dtype=int))        
        
        # If want 1:1 for projection of excitatory onto inhibitory neurons
        #c_rows = list(arange(0,p_couple2,dtype=int))
        #c_cols = list(arange(N-i_couple,N,dtype=int))
        
        # If want 1:! for projection of inhibitory onto excitatory neurons
        #c_rows = list(arange(N-p_couple2,N,dtype=int))
        #c_cols = list(arange(0,p_couple2,dtype=int))
        
        S3.connect(i=c_rows, j=c_cols) # Manually defined coupling
        S3.w = w_couple
        ###################################
        
        ##### Probabilistic coupling #####
        #S3.connect(p=0.05) # Probabilistic connection - Chance that G2 will connect with and spike from G1
        #S3.w = 0.02
        #S3.connect(p=p_couple)
        ###################################
                
        # Coupling matrix
        coup_mat = [[0 for x in range(N)] for y in range(N)]

        for ii in range(len(c_rows)):
            for jj in range(len(c_cols)):
                coup_mat[ii][ii] = 1      # Matrix has 1s for connections and 0s for none

        statemon1 = StateMonitor(G1, 'v', record=0) # Records just neuron 0 to save resources
        spikemon1 = SpikeMonitor(G1, variables='v')
        statemon2 = StateMonitor(G2, 'v', record=0) # Records just neuron 0 to save resources
        spikemon2 = SpikeMonitor(G2, variables='v')
                
        run(run_time*ms, 'text')

        return statemon1,spikemon1,statemon2,spikemon2,c_rows,c_cols,coup_mat
        
    def spike_time(self,spikemon):
        all_values = spikemon.all_values()
        spike_times = all_values['t'][0] # Spike times for just neuron 0
        all_spikes = spikemon.t/ms # Spike times for all neurons
        
        return spike_times,all_spikes
        
    def voltage_monitor(self,statemon):
        plot(statemon.t/ms, statemon.v[0])
        #plot(statemon.t/ms, statemon.v[1])  # Plots second neuron      
        ylabel('Voltage (V)')
        xlabel('Time (ms)')
        
    def raster_plot(self,spikemon,spikemon_other):
        #ion()
        plot(spikemon.t/ms, spikemon.i, '.r')
        plot(spikemon_other.t/ms, spikemon_other.i, '.k') # Plots overlay of each network
        xlabel('Time (ms)')
        ylabel('Neuron index');
        #plt.show(block=True)
        
    def spike_hist(self,run_time,all_spikes):
        my_bins = arange(0,run_time+2,2)
        plt.hist(all_spikes, bins=my_bins)
        xlabel('Time (ms)')
        ylabel('Total number of spikes')

In [ ]:
'''
Specifications of Network 1

Single network of 100 neurons (80 excitatory/20 inhibitory)
Random topology, using the simplified neuron model of last year's code
Total run time of 1,000 ms

Display the network model with the functions of the Visualization class
Plot the raster plot of the simulation using the functions of the BrianVisualization class. 

The Sample 1 and Sample 4 code from last year contain most of the code to start this model of a single network.  
Please comp'''


import networkx as nx
import brian2
import numpy as np

from brian2 import*
%matplotlib inline 

'''
Sample function 1 calling
Change parameters to fit neural model
Computes weighted adjacency matrix

Parameters to be user-defined:
    n: nodes
    m: edges
    k: each node is connected to k nearest neightbors
    p: probability of adding new edge for each edge
    d: degree of each node
'''
# Sample call function
# Define 100 neurons
n = 100 




m = 400
k = 2
p = 0.2
d = 2

rand_seed = np.random.seed(int(time.time())) # To seed random number generator based on time

Adj = AdjacencyMatrix(n) # Initiates...only runs the '__ini__' function at this point

#[A,G] = Adj.all_to_all(n) # Defines all-to-all topology
[A,G] = Adj.random(n,m) # Defines random topology
#[A,G] = Adj.small_world(n,k,p) # Defines small-world topology
#[A,G] = Adj.regular(d,n) # Defines regular topology
#[A,G] = Adj.scale_free(n) # Defines scale-free topology

W,rows,cols,connect_W,new_coord,A_temp5,myW = Adj.Weighted(A,n) # Output






